Default properties

Project crs: "EPSG:31370"

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import Point, Polygon, LineString, MultiLineString, MultiPoint
from shapely.wkt import loads
from shapely.ops import nearest_points
from shapely import wkt
import shapely.wkt

import random

import fiona

In [2]:
# PROJ_CRS = 'EPSG:31370'

In [3]:
def load_data(path):
    """
    Loads the data from the given path, 
    and prints the shape and crs of the data.
    """
    data = gpd.read_file(path)
    print(data.shape)
    proj_crs = data.crs
    print("Set Project crs to:", proj_crs)
    return data, proj_crs

In [4]:
pwd

'c:\\workdir\\develop\\gopeg\\preprocessing\\notebooks'

In [5]:
import os
os.chdir('../')

In [6]:
bxl_water, PROJ_CRS = load_data(r'data\data_preprocess\bxl_networks\bxk_hydronetwork.shp') #, crs=PROJ_CRS)

(83, 35)
Set Project crs to: epsg:31370


## Add begin and end points to the water network

In [7]:
#Check for multiline strings in a dataset
def check_multiline(df):
    """This function checks for multiline strings
        from the geometry column in a given dataset"""
    lst = df['geometry'].to_list()
    multiline_count = 0
    for item in lst:
        if isinstance(item, MultiLineString):
            multiline_count += 1
    print("MultiLinesStrings:" , multiline_count)
    

#filter out multilinestring dataset
def multiline_to_linestring(df):
    #filter out multilinestring dataset
    multiline_df = df[df['geometry'].apply(lambda x: isinstance(x, MultiLineString))]
    linestrings_df = df[df.geom_type == 'LineString']
    if len(linestrings_df) == len(df):
        print("No multiline strings found")
        return df

    else:
        print("Checking for multiline strings...")
        check_multiline(df)
        # turn multilinestrings into linestrings
        linestrings = []
        for idx, row in multiline_df.iterrows():
            inlines = row.geometry
            outcoords = [list(item.coords) for item in inlines]
            outline = shapely.geometry.LineString([i for sublist in outcoords for i in sublist])
            #outline_geom = shapely.wkt.dumps(outline)
            linestrings.append(outline)

            
        # add  linestrings to dataframe and drop original geom column
        multiline_df['exploded'] = linestrings
        multiline_df = multiline_df.drop(['geometry'], axis=1).rename(columns={'exploded': 'geometry'}).reset_index(drop=True)
        multiline_gdf = gpd.GeoDataFrame(multiline_df, geometry='geometry', crs=PROJ_CRS)

        gdf = linestrings_df.append(multiline_gdf).reset_index(drop=True)
        print("Checking for multiline strings after...") 
        check_multiline(gdf)

    return gdf

In [8]:
bxl_water_df = multiline_to_linestring(bxl_water)

Checking for multiline strings...
MultiLinesStrings: 6
Checking for multiline strings after...
MultiLinesStrings: 0


C:\Users\u0119822\AppData\Local\Temp\ipykernel_18764\750445056.py:29: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  outcoords = [list(item.coords) for item in inlines]
c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__se

In [9]:
bxl_water_df.columns

Index(['OBJECTID', 'DFDD', 'RN_I_ID', 'REX', 'HYP', 'LOC', 'FUN', 'NVS',
       'LENGTH', 'TR', 'LONGPATH', 'CUM_LEN', 'PENTE', 'CGNELIN', 'BEGLIFEVER',
       'ENDLIFEVER', 'UPDAT_BY', 'UPDAT_WHEN', 'ERM_ID', 'MC', 'MONOT_Z',
       'LENGTH_GEO', 'INSPIRE_ID', 'thematicId', 'OBJECT_ID', 'TNODE',
       'STRAHLER', 'nameTxtInt', 'nameText', 'NEXTUPID', 'NEXTDOWNID', 'FNODE',
       'CatchID', 'PFAFSTETTE', 'geometry'],
      dtype='object')

In [10]:
def add_beginpoints(df):
    startnodes_gdf = df.copy()
    lst = startnodes_gdf['geometry'].to_list()
    beginpoints = []
    for item in lst:
        first = Point(item.coords[0])
        first_precise = shapely.wkt.dumps(first)
        beginpoints.append(first_precise)

    startnodes_gdf['start_point'] = [wkt.loads(g) for g in beginpoints]
    startnodes_gdf = startnodes_gdf.drop(['geometry'], axis=1).rename(columns={'start_point': 'geometry'})
    
    startnodes_gdf = gpd.GeoDataFrame(startnodes_gdf, geometry=startnodes_gdf['geometry'], crs=PROJ_CRS) #.drop(columns=[col])
    return startnodes_gdf


def add_endpoints(df):
    endnodes_gdf = df.copy()
    lst = endnodes_gdf['geometry'].to_list()
    endpoints = []
    for item in lst:
        last = Point(item.coords[-1])
        last_precise = shapely.wkt.dumps(last)
        endpoints.append(last_precise)

    endnodes_gdf['end_point'] = [wkt.loads(g) for g in endpoints]
    endnodes_gdf = endnodes_gdf.drop(['geometry'], axis=1).rename(columns={'end_point': 'geometry'})

    endnodes_gdf = gpd.GeoDataFrame(endnodes_gdf, geometry=endnodes_gdf['geometry'], crs=PROJ_CRS) #.drop(columns=[col])
    return endnodes_gdf

In [11]:
startnodes_gdf = add_beginpoints(bxl_water_df)
endnodes_gdf = add_endpoints(bxl_water_df)
print(startnodes_gdf.shape)
print(endnodes_gdf.shape)

(83, 35)
(83, 35)


c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [12]:
def get_nodes_geometry(df): 
    startnodes_gdf = add_beginpoints(df)
    endnodes_gdf = add_endpoints(df)   
    nodes_geom = pd.merge(startnodes_gdf[['FNODE','geometry']], endnodes_gdf[['TNODE','geometry']], on='geometry', how='outer').reset_index()

    #drop geometry duplicates and leave only one node geometry
    nodes_gdf = (nodes_geom.drop_duplicates(subset=['geometry'], keep='first')
                        .drop(columns=['index'])
                        .reset_index(drop=True))

    #assign unique ids to the nodes              
    start_id = nodes_gdf[nodes_gdf['TNODE'].isnull()].drop(columns=['TNODE']).rename(columns={'FNODE': 'node_id'})
    end_id = nodes_gdf[nodes_gdf['FNODE'].isnull()].drop(columns=['FNODE']).rename(columns={'TNODE': 'node_id'})
    other_nodes = nodes_gdf[nodes_gdf['FNODE'].notnull() & nodes_gdf['TNODE'].notnull()].drop(columns=['TNODE']).rename(columns={'FNODE': 'node_id'})
    df_list = [start_id, end_id, other_nodes]
    final_nodes_gdf = pd.concat(df_list)
    return final_nodes_gdf

In [13]:
nodes_gdf = get_nodes_geometry(bxl_water_df)
water_nodes_df = nodes_gdf.reset_index(drop=True)

c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [14]:
water_nodes_df.head()

,node_id,geometry
0,NO25019914,POINT (149013.615 164578.480)
1,NO25019967,POINT (145305.631 166954.356)
2,NO25020010,POINT (145049.456 166801.083)
3,NO25025475,POINT (142015.321 168681.806)
4,NO25019997,POINT (153488.669 164773.036)


In [15]:
water_nodes_df.loc[20, 'node_id'] = 'NO25019967_1'

In [16]:
water_nodes_df.loc[20]

node_id                                      NO25019967_1
geometry    POINT (145889.71414541386 165280.08683214616)
Name: 20, dtype: object

In [17]:
water_nodes_df[water_nodes_df['node_id'] == 'NO25019967']

,node_id,geometry
1,NO25019967,POINT (145305.631 166954.356)


In [18]:
bxl_water_df.columns

Index(['OBJECTID', 'DFDD', 'RN_I_ID', 'REX', 'HYP', 'LOC', 'FUN', 'NVS',
       'LENGTH', 'TR', 'LONGPATH', 'CUM_LEN', 'PENTE', 'CGNELIN', 'BEGLIFEVER',
       'ENDLIFEVER', 'UPDAT_BY', 'UPDAT_WHEN', 'ERM_ID', 'MC', 'MONOT_Z',
       'LENGTH_GEO', 'INSPIRE_ID', 'thematicId', 'OBJECT_ID', 'TNODE',
       'STRAHLER', 'nameTxtInt', 'nameText', 'NEXTUPID', 'NEXTDOWNID', 'FNODE',
       'CatchID', 'PFAFSTETTE', 'geometry'],
      dtype='object')

In [19]:
#startnodes_gdf.to_file(r'..\data_preprocess\bxl_networks\bxl_startnodes.shp')
#endnodes_gdf.to_file(r'..\data_preprocess\bxl_networks\bxl_endnodes.shp')

In [20]:
#nodes_gdf.to_file(r'..\data_transform\bxl_water_nodes.shp')

## Join water and sewer network

In [21]:
bxl_water_df.iloc[0]

OBJECTID                                                  20197
DFDD                                                      BH140
RN_I_ID                                                    None
REX                                                          BE
HYP                                                         NaN
LOC                                                         NaN
FUN                                                         NaN
NVS                                                         NaN
LENGTH                                              2637.439941
TR                                                           NA
LONGPATH                                           191619.03125
CUM_LEN                                             2637.436768
PENTE                                                       NaN
CGNELIN                                                     NaN
BEGLIFEVER                              2020/03/30 00:00:00.000
ENDLIFEVER                              

In [22]:
assert bxl_water_df.OBJECTID.nunique() == bxl_water_df.OBJECT_ID.nunique()

## Load discharge points

DIscharge points are used to connect a sewer network to a water network.

In [27]:
bxl_disharge_pts, PROJ_CRS = load_data(r'data\data_preprocess\bxl_networks\bxl_uwwtd_discharge.shp')

(2, 33)
Set Project crs to: epsg:31370


In [28]:
bxl_disharge_pts

,dcpState,repCode,uwwCode,dcpCode,dcpName,dcpNuts,dcpLatitud,dcpLongitu,dcpWaterBo,dcpIrrigat,...,dcpRecei_1,dcpWFDSu_1,dcpWFDRBDR,dcpBeginLi,dcpEndLife,dcpRemarks,dcpGeometr,dcpDischar,rptMStateK,geometry
0,1.0,BE-2018,BEBRU1,BEBRU1,DISCHARGE POINT BRU1,BEZZZ,50.8145,4.30278,FW,None,...,None,None,None,2000-08-01,None,None,E6100000010C1288D7F50B361140C74B378941684940,3617633.0,BE,POINT (145350.666 167119.510)
1,1.0,BE-2018,BEBRU2,BEBRU2,DISCHARGE POINT BRU2,BEZZZ,50.9040,4.41074,FW,None,...,None,None,None,2007-03-03,None,None,E6100000010CDBA2CC0699A41140C1CAA145B6734940,3617634.0,BE,POINT (152953.286 177074.438)


In [29]:
bxl_disharge_pts.iloc[0]

dcpState                                                1.0
repCode                                             BE-2018
uwwCode                                              BEBRU1
dcpCode                                              BEBRU1
dcpName                                DISCHARGE POINT BRU1
dcpNuts                                               BEZZZ
dcpLatitud                                          50.8145
dcpLongitu                                          4.30278
dcpWaterBo                                               FW
dcpIrrigat                                             None
dcpTypeOfR                                              A54
rcaCode                                                 BE1
dcpSurface                                              1.0
dcpWater_1                                BE_Senne_Zenne_BR
dcpNotAffe                                                0
dcpMSProvi                                                0
dcpCOMAcce                              

In [30]:
#drop_cols_pts = ['dcpWater_1', 'dcpNuts', 'dcpReceivi', 'dcpWFDRBD','uwwCode', 'dcpWFDSubU', 'rcaCode', 'dcpDischar', 'repCode', 'dcpLatitud', 'dcpLongitu', 'dcpSurface', 'dcpTypeOfR', 'dcpIrrigat', 'dcpNotAffe', 'dcpMSProvi', 'dcpCOMAcce', 'dcpGroundW', 'dcpWater_2', 'dcpGroun_1', 'dcpWFDSu_1', 'dcpWFDRBDR','dcpRemarks', 'dcpGeometr', 'rptMStateK']
#discharge_pts_df = bxl_disharge_pts.drop(drop_cols_pts, axis=1)

In [31]:
discharge_pts_df = bxl_disharge_pts.loc[:, ['dcpState', 'dcpCode', 'dcpName', 'dcpWaterBo', 'dcpRecei_1', 'dcpBeginLi', 'dcpEndLife', 'geometry']]
    

In [32]:
discharge_pts_df

,dcpState,dcpCode,dcpName,dcpWaterBo,dcpRecei_1,dcpBeginLi,dcpEndLife,geometry
0,1.0,BEBRU1,DISCHARGE POINT BRU1,FW,None,2000-08-01,None,POINT (145350.666 167119.510)
1,1.0,BEBRU2,DISCHARGE POINT BRU2,FW,None,2007-03-03,None,POINT (152953.286 177074.438)


## Find nearest point on network

### 1. Convert the data into the same crs

In [33]:
gdf_p = discharge_pts_df.to_crs(PROJ_CRS)
print(gdf_p.crs)
gdf_l = bxl_water_df.to_crs(PROJ_CRS)
print(gdf_l.crs)

epsg:31370
epsg:31370


### 2. Join the two datasets to the nearest geometries

In [34]:
water_discharge_df = gpd.sjoin_nearest(discharge_pts_df, bxl_water_df).merge(bxl_water_df['geometry'], left_on="index_right", right_index=True)

#df_n["distance"] = df_n.apply(lambda r: r["geometry_x"].distance(r["geometry_y"]), axis=1)

In [35]:
water_discharge_df.sample(2)

,dcpState,dcpCode,dcpName,dcpWaterBo,dcpRecei_1,dcpBeginLi,dcpEndLife,geometry_x,index_right,OBJECTID,...,TNODE,STRAHLER,nameTxtInt,nameText,NEXTUPID,NEXTDOWNID,FNODE,CatchID,PFAFSTETTE,geometry_y
0,1.0,BEBRU1,DISCHARGE POINT BRU1,FW,None,2000-08-01,None,POINT (145350.666 167119.510),1,20350,...,NO25020064,4.0,None,ZENNE I,RL25020250,RL25020355,NO25019967,25,None,"LINESTRING (145305.631 166954.356, 145305.983 ..."
1,1.0,BEBRU2,DISCHARGE POINT BRU2,FW,None,2007-03-03,None,POINT (152953.286 177074.438),48,20829,...,NO25020533,1.0,SENNE-ZENNE,ZENNE,None,RL25020839,NO25020532,25,None,"LINESTRING (150482.760 174395.771, 150486.285 ..."


In [36]:
#gdf.to_file(r'D:\SADL\geopackage\wal_layers.gpkg', layer='wal_disharge_pts2', driver='GPKG')

### 3. Project discharge point to nearest point on water network

In [81]:
# def get_nearest_point(df, line_col, point_col):
#     """
#     For each point in points_df, find the nearest point in lines_df.
#     """
#     geoms = []
#     for idx, row in df.iterrows():
#         destinations = MultiPoint(row[line_col].coords) #geometry_y
#         print(type(destinations))
#         nearest_geoms = nearest_points(row[point_col], destinations) #geometry_x
#         try:
#             for coord in destinations:
#                 if coord == nearest_geoms[1]:
#                     geoms.append(coord)
#         except ValueError:
#             print("No nearest point found for {}".format(row.CODEKOPPNT))
#     return geoms

In [110]:
def get_nearest_point(df, line_col, point_col):
    """
    For each point in points_df, find the nearest point in lines_df.
    """
    geoms = []
    for idx, row in df.iterrows():
        destinations = MultiPoint(np.array(row[line_col].coords)) #geometry_y
        # print(type(destinations))
        # source = np.array(row[point_col])
        nearest_geoms = nearest_points(row[point_col], destinations) #geometry_x
        try:
            for coord in destinations.geoms:
                if coord == nearest_geoms[1]:
                    geoms.append(coord)
        except ValueError:
            print("No nearest point found for {}".format(row.point_col))
    return geoms

In [111]:
# %%timeit
test_nodes = get_nearest_point(water_discharge_df, 'geometry_y', 'geometry_x')

In [112]:
water_discharge_df['connection_nodes'] = get_nearest_point(water_discharge_df, 'geometry_y', 'geometry_x')

c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [39]:
water_discharge_df.head(2)

,dcpState,dcpCode,dcpName,dcpWaterBo,dcpRecei_1,dcpBeginLi,dcpEndLife,geometry_x,index_right,OBJECTID,...,STRAHLER,nameTxtInt,nameText,NEXTUPID,NEXTDOWNID,FNODE,CatchID,PFAFSTETTE,geometry_y,connection_nodes
0,1.0,BEBRU1,DISCHARGE POINT BRU1,FW,None,2000-08-01,None,POINT (145350.666 167119.510),1,20350,...,4.0,None,ZENNE I,RL25020250,RL25020355,NO25019967,25,None,"LINESTRING (145305.631 166954.356, 145305.983 ...",POINT (145345.0987288037 167120.24663728196)
1,1.0,BEBRU2,DISCHARGE POINT BRU2,FW,None,2007-03-03,None,POINT (152953.286 177074.438),48,20829,...,1.0,SENNE-ZENNE,ZENNE,None,RL25020839,NO25020532,25,None,"LINESTRING (150482.760 174395.771, 150486.285 ...",POINT (152942.29463238214 177058.22313042078)


In [40]:
nodes_df = water_discharge_df[['OBJECT_ID', 'dcpCode', 'connection_nodes']]\
                            .rename(columns={'connection_nodes': 'geometry'})
connection_nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry='geometry', crs=PROJ_CRS)

water_df = water_discharge_df[['OBJECT_ID', 'thematicId', 'nameText', 'geometry_y']]\
                            .rename(columns={'geometry_y': 'geometry'})
water_gdf = gpd.GeoDataFrame(water_df, geometry='geometry', crs=PROJ_CRS)

In [41]:
connection_nodes_gdf


,OBJECT_ID,dcpCode,geometry
0,RL25020350,BEBRU1,POINT (145345.099 167120.247)
1,RL25020829,BEBRU2,POINT (152942.295 177058.223)


In [42]:
#connection_nodes_upload = connection_nodes_gdf.drop(columns=['coords'])
#connection_nodes_gdf.to_file(r'data_transform\bxl_connection_nodes.shp')

In [43]:
water_discharge_df.head(2)

,dcpState,dcpCode,dcpName,dcpWaterBo,dcpRecei_1,dcpBeginLi,dcpEndLife,geometry_x,index_right,OBJECTID,...,STRAHLER,nameTxtInt,nameText,NEXTUPID,NEXTDOWNID,FNODE,CatchID,PFAFSTETTE,geometry_y,connection_nodes
0,1.0,BEBRU1,DISCHARGE POINT BRU1,FW,None,2000-08-01,None,POINT (145350.666 167119.510),1,20350,...,4.0,None,ZENNE I,RL25020250,RL25020355,NO25019967,25,None,"LINESTRING (145305.631 166954.356, 145305.983 ...",POINT (145345.0987288037 167120.24663728196)
1,1.0,BEBRU2,DISCHARGE POINT BRU2,FW,None,2007-03-03,None,POINT (152953.286 177074.438),48,20829,...,1.0,SENNE-ZENNE,ZENNE,None,RL25020839,NO25020532,25,None,"LINESTRING (150482.760 174395.771, 150486.285 ...",POINT (152942.29463238214 177058.22313042078)


## Project and Split Lines

In [44]:
def get_line_coords(line):

    """ Returns a list of tuples of coordinates"""
    
    coords_list = []
    multi_points = MultiPoint(line.coords)

    #multi_points_list = [shapely.wkt.dumps(g) for g in multi_points]
    #multi_points_geoms = [shapely.wkt.loads(i) for i in multi_points_list]
    for i in multi_points:
        
        long, lat = i.x, i.y
 
        coords_list.append((long, lat))

    return coords_list

def get_point_coords(gdf):
    return gdf.geometry.apply(lambda geom: (geom.x, geom.y))

In [45]:
connection_nodes_gdf['coords'] = get_point_coords(connection_nodes_gdf)
water_gdf['coords'] = water_gdf.apply(lambda row: get_line_coords(row.geometry), axis=1)

C:\Users\u0119822\AppData\Local\Temp\ipykernel_18764\3869889012.py:10: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for i in multi_points:


In [46]:
def get_line_segments(l, points_list):

    idx_list = [i for i, item in enumerate(l) if item in points_list] # compares the two lists and returns the indexes of occurence
    
    p = [l[i] for i in idx_list] # get correct order of points list on the line

    super_list = []

    start_idx = 0



    #print("Index list: ", idx_list)

    if len(idx_list) == 1 and (p[0] == l[0] or p[0] == l[-1]): #      (i == 0 or i == len(l)-1) and len(idx_list) == 1:
        #print("One split point, at first or last index")
        line_segment = LineString(l)
        super_list.append(line_segment)

    elif len(idx_list) == 2 and (p[0] == l[0] or p[1] == l[-1]):
        #print("Two split points, at first and last index")
        line_segment = LineString(l)
        super_list.append(line_segment)

    else:

        #import pdb; pdb.set_trace()
        for i in idx_list:
            # In the case of the first coordinates of a line being a split point but there are other split points
            if i == 0 and len(idx_list) > 1:
                index_list = len(idx_list)
                
                #print(f"First index is a split point, with {index_list} split points")
                continue

            else:
                #print("Many split points")
                stop_idx = i + 1 #grab list elements until index i
                #print(f"stop index is {stop_idx}")

                line_list = l[start_idx: stop_idx]
                line_segment = LineString(line_list)
                super_list.append(line_segment)
                start_idx = i #reset the start index to the number of the prevous stop index

                #Catch if a split point is at the end of the line list
                #if stop_idx == len(l) or p[-1] == line_list[-1]:
                    #print("Many split points, with a split point at end of list") # stop index goes beyond the line list
                #    break

                 #super list still has one more segment to add
                if len(super_list) == len(idx_list):
                    #print("Add last segment after last split point") # stop index goes beyond the line list
                    #super list still has one more segment to add
                    ##print("Add last segment after last split point") # stop index goes beyond the line list
                    #print("super list still has one more segment to add")
                    #print(f"stop index is {stop_idx}")
                    #print(f'Linestring length {len(l)}')
                    #print(f'Index list {len(idx_list)}')
                    #print(f'Length of superlist {len(super_list)}')
                    #print('Last sgment is "last_segment = l[stop_idx-1:len(l)]"')
                    #n = len(l) - len(super_list)

                    last_segment = l[stop_idx-1:len(l)]
                    if stop_idx == len(l):
                        #print("Split point at end of list") # stop index goes beyond the line list
                        break
                    # n = len(l) - len(super_list)
                    #last_segment = l[stop_idx-1:len(l)] # Grab the last segments of the list from the prevous stop_idx-1, to the end of the lin len(l)
                    else:
                        #print("Split point at end of list")
                        last_segment_geom = LineString(last_segment)
                        super_list.append(last_segment_geom)
                    
                    #if stop_idx == len(l) or p[-1] == line_list[-1]:
                        #print("Split point at end of list") # stop index goes beyond the line list
                        #break

        #if points_list[0] == line_list[0] and points_list[-1] == line_list[-1]:
        #    assert len(super_list) == len(idx_list) - 1
        #elif points_list[0] == line_list[0] or points_list[-1] == line_list[-1]:
        #    assert len(super_list) == len(idx_list)
        #else:
        #    assert len(super_list) == len(idx_list) + 1

    return super_list

# pass a dataframe to the function

def split_lines(water_gdf, nodes_gdf, unique_id):

    #nodes_no_duplicates = nodes_gdf.drop_duplicates(subset='CODEKOPPNT')
    water_no_duplicates = water_gdf.drop_duplicates(subset=unique_id)

    groups = nodes_gdf.groupby(unique_id)

    codes_list = nodes_gdf[unique_id].to_list()
    unique_code_list = list(set(codes_list))
    #print(len(unique_code_list))

    #print(unique_code_list[71:73])

    all_segments = []
    ids = []
    #counter = 0

    for num, i in enumerate(unique_code_list):
        #print(f"Start loop {num, i}")
        
        #group = groups.get_group(i)
        #print("Get points list")

        points_list = groups.get_group(i).coords.to_list()
        #print("Points list: ", points_list)
        
        #print(points_list)

        line = water_no_duplicates[water_no_duplicates[unique_id] == i]['coords'][:1]
        #indx = water_no_duplicates[water_no_duplicates[unique_id] == i].index [0]

        points_list = groups.get_group(i).coords.to_list()
        
        line_segments = get_line_segments(*line, points_list)
        num_segments = len(line_segments)

        all_segments.extend(line_segments)
        #ids.append(indx)
        num_unique_ids = [i] * num_segments
        #get group for each unique code
        #group_ids = groups.get_group(i)[unique_id].to_list()

        #assert len(flat_list) == len(water_no_duplicates)
        ids.extend(num_unique_ids)
    
        #print(f"Line: {i}")
        #print(f'len({line_segments}) line_segments added')
        #print(line_segments)

        #all_segments = all_segments.extend(line_segment)

    #print(all_segments)
        #counter += 1

    #print(unique_code_list[72:74])
    ## Create dataframe with all segments
    print(len(all_segments))
    print(len(ids))
    #ids_list = list(range(len(ids)))
    gdf_segments = gpd.GeoDataFrame(list(range(len(all_segments))), geometry=all_segments, crs=PROJ_CRS)
    #gdf_segments = gpd.GeoDataFrame(gdf_segments, geometry='geometry', crs=PROJ_CRS)
    gdf_segments.columns = ['index', 'geometry']
    gdf_segments[unique_id] = ids
    return gdf_segments

In [47]:
import time

initialTime = time.time()
splitlines_df = split_lines(water_gdf, connection_nodes_gdf, 'OBJECT_ID')
finishTime = time.time()
print(f"Time taken: {finishTime - initialTime}")
print("********************************************************")

4
4
Time taken: 0.029994487762451172
********************************************************


In [48]:
print(splitlines_df.shape)
splitlines_df

(4, 3)


,index,geometry,OBJECT_ID
0,0,"LINESTRING (145305.631 166954.356, 145305.983 ...",RL25020350
1,1,"LINESTRING (145345.099 167120.247, 145350.687 ...",RL25020350
2,2,"LINESTRING (150482.760 174395.771, 150486.285 ...",RL25020829
3,3,"LINESTRING (152942.295 177058.223, 152961.803 ...",RL25020829


## Combine all nodes

In [49]:
water_nodes_df['source'] = 'water_node'
water_nodes_df.head(2)


,node_id,geometry,source
0,NO25019914,POINT (149013.615 164578.480),water_node
1,NO25019967,POINT (145305.631 166954.356),water_node


In [50]:
connection_nodes_gdf

,OBJECT_ID,dcpCode,geometry,coords
0,RL25020350,BEBRU1,POINT (145345.099 167120.247),"(145345.0987288037, 167120.24663728196)"
1,RL25020829,BEBRU2,POINT (152942.295 177058.223),"(152942.29463238214, 177058.22313042078)"


In [51]:
connection_nodes = connection_nodes_gdf[['dcpCode', 'geometry']].rename(columns={'dcpCode': 'node_id'})
connection_nodes['source'] = 'connection_node'
assert connection_nodes.geometry.nunique() == connection_nodes.node_id.nunique()
print(connection_nodes.shape)
connection_nodes.head()

(2, 3)


,node_id,geometry,source
0,BEBRU1,POINT (145345.099 167120.247),connection_node
1,BEBRU2,POINT (152942.295 177058.223),connection_node


In [52]:
bxl_nodes_combined = pd.concat([water_nodes_df, connection_nodes])

final_nodes_combined = bxl_nodes_combined.drop_duplicates(subset='geometry', keep='last').reset_index(drop=True) #keep last keeps the connection nodes over the water nodes
assert len(final_nodes_combined) == bxl_nodes_combined.geometry.nunique()

In [53]:
final_nodes_combined

,node_id,geometry,source
0,NO25019914,POINT (149013.615 164578.480),water_node
1,NO25019967,POINT (145305.631 166954.356),water_node
2,NO25020010,POINT (145049.456 166801.083),water_node
3,NO25025475,POINT (142015.321 168681.806),water_node
4,NO25019997,POINT (153488.669 164773.036),water_node
...,...,...,...
85,NO25020541,POINT (152941.090 177439.677),water_node
86,NO25020542,POINT (152909.773 177457.185),water_node
87,NO25019899,POINT (146845.939 164060.517),water_node
88,BEBRU1,POINT (145345.099 167120.247),connection_node


In [54]:
final_nodes_combined[final_nodes_combined['node_id'] == 'NO25019967']

,node_id,geometry,source
1,NO25019967,POINT (145305.631 166954.356),water_node


In [55]:
final_nodes_combined.node_id.nunique()

90

In [56]:
#add a sewer node column into the connection nodes

def add_sewernode_id(row):
    if row['source'] == 'connection_node':
        return row['node_id']
    else:
        return None

In [57]:
def get_water_nodes(prefix):
    nodes_all = final_nodes_combined.copy()
    nodes_all['sewernode_id'] = nodes_all.apply(add_sewernode_id, axis=1)

    conn_df = nodes_all[nodes_all['source'] == 'connection_node']
    water_nodes = nodes_all.loc[nodes_all['source'] == 'water_node']

    nodes_list = water_nodes['node_id'].to_list()
    start_num = max([int(i[2:]) for i in nodes_list])

    diff = len(nodes_all.index) - len(water_nodes_df.index)

    conn_df['node_id'] = range((start_num + 1), (start_num + diff +1))
    conn_df['node_id'] = 'NO' + conn_df['node_id'].astype(str)

    nodes_all = pd.concat([water_nodes, conn_df])
    nodes_all_gdf = gpd.GeoDataFrame(nodes_all, geometry='geometry', crs=PROJ_CRS)

    return nodes_all_gdf

In [58]:
waternodes = get_water_nodes('NO')

c:\Users\u0119822\.conda\envs\gopeg\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [59]:
waternodes

,node_id,geometry,source,sewernode_id
0,NO25019914,POINT (149013.615 164578.480),water_node,None
1,NO25019967,POINT (145305.631 166954.356),water_node,None
2,NO25020010,POINT (145049.456 166801.083),water_node,None
3,NO25025475,POINT (142015.321 168681.806),water_node,None
4,NO25019997,POINT (153488.669 164773.036),water_node,None
...,...,...,...,...
85,NO25020541,POINT (152941.090 177439.677),water_node,None
86,NO25020542,POINT (152909.773 177457.185),water_node,None
87,NO25019899,POINT (146845.939 164060.517),water_node,None
88,NO250199672,POINT (145345.099 167120.247),connection_node,BEBRU1


In [60]:
discharge_pts_df

,dcpState,dcpCode,dcpName,dcpWaterBo,dcpRecei_1,dcpBeginLi,dcpEndLife,geometry
0,1.0,BEBRU1,DISCHARGE POINT BRU1,FW,None,2000-08-01,None,POINT (145350.666 167119.510)
1,1.0,BEBRU2,DISCHARGE POINT BRU2,FW,None,2007-03-03,None,POINT (152953.286 177074.438)


In [61]:
merge_waternodes_sewer = pd.merge(waternodes, discharge_pts_df[['dcpCode', 'dcpName', 'dcpWaterBo', 'dcpState']], left_on='sewernode_id', right_on='dcpCode', how='left')

In [62]:
merge_waternodes_sewer

,node_id,geometry,source,sewernode_id,dcpCode,dcpName,dcpWaterBo,dcpState
0,NO25019914,POINT (149013.615 164578.480),water_node,None,NaN,NaN,NaN,NaN
1,NO25019967,POINT (145305.631 166954.356),water_node,None,NaN,NaN,NaN,NaN
2,NO25020010,POINT (145049.456 166801.083),water_node,None,NaN,NaN,NaN,NaN
3,NO25025475,POINT (142015.321 168681.806),water_node,None,NaN,NaN,NaN,NaN
4,NO25019997,POINT (153488.669 164773.036),water_node,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
85,NO25020541,POINT (152941.090 177439.677),water_node,None,NaN,NaN,NaN,NaN
86,NO25020542,POINT (152909.773 177457.185),water_node,None,NaN,NaN,NaN,NaN
87,NO25019899,POINT (146845.939 164060.517),water_node,None,NaN,NaN,NaN,NaN
88,NO250199672,POINT (145345.099 167120.247),connection_node,BEBRU1,BEBRU1,DISCHARGE POINT BRU1,FW,1.0


In [64]:
#merge_waternodes_sewer.to_file(r"data_transform\final_bxl_nodes_combined.shp")

## Get start and end IDs for the line segments, to include new connection nodes

In [65]:
def line_segments_start_end_ids(splitlines_df, line_id, all_nodes_gdf, node_id):
    """ Returns a dataframe with the start and end ids of each line segment """
    #splitlines_df = splitlines_df.drop(columns=['index'])

    splitlines_df['coords'] = splitlines_df.apply(lambda row: get_line_coords(row.geometry), axis=1)
    all_nodes_gdf['coords'] = get_point_coords(all_nodes_gdf)
    #join linestrings to the nearest node, in this case the node attached to the line
    joined_lines_nodes = gpd.sjoin_nearest(splitlines_df, all_nodes_gdf, how='left').reset_index()
    

    idx_start = []
    start_id = []
    idx_end = []
    end_id = []
    for idx, row in joined_lines_nodes.iterrows():
        if row.coords_right == row.coords_left[0]:
            idx_start.append(row['index'])
            start_id.append(row[node_id])
        elif row.coords_right == row.coords_left[-1]:
            idx_end.append(row['index'])
            end_id.append(row[node_id])


    start_id_df = pd.DataFrame({'line_index': idx_start, f'start_{node_id}': start_id})
    end_id_df = pd.DataFrame({'line_index': idx_end, f'end_{node_id}': end_id})
    
    
    start_id_df_all = pd.merge(start_id_df, joined_lines_nodes[['index', node_id, line_id, 'geometry']], left_on='line_index', right_on = 'index', how='left').drop_duplicates('geometry')
    end_id_df_all = pd.merge(end_id_df, joined_lines_nodes[['index',  node_id, line_id, 'geometry']], left_on='line_index', right_on = 'index', how='left').drop_duplicates('geometry')

    
    merged_start_end_df = pd.merge(start_id_df_all, end_id_df_all, on='geometry', how='outer')
    merged_start_end_df = gpd.GeoDataFrame(merged_start_end_df, geometry='geometry', crs=PROJ_CRS)

    assert merged_start_end_df[f'{line_id}_x'].all() == merged_start_end_df[f'{line_id}_y'].all()

    splitlines_with_ids = merged_start_end_df.drop(['line_index_x', 'index_x', 'node_id_x',	'index_y', 'line_index_y', 'node_id_y', f'{line_id}_y'], axis=1)\
                                            .rename(columns={'start_node_id': 'start_ID', 'end_node_id': 'end_ID'})

    assert len(splitlines_with_ids) == splitlines_with_ids.geometry.nunique()

    return splitlines_with_ids

In [66]:
node_id = 'node_id'
line_id = 'OBJECT_ID'
#bxl_water_df = bxl_water_df.rename(columns={'FNODE': 'start_ID', 'TNODE': 'end_ID'})
splitlines_with_ids = line_segments_start_end_ids(splitlines_df, line_id, waternodes, node_id)
splitlines_with_ids = splitlines_with_ids.rename(columns={'start_ID': 'FNODE', 'end_ID': 'TNODE'})

C:\Users\u0119822\AppData\Local\Temp\ipykernel_18764\3869889012.py:10: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for i in multi_points:


In [67]:
splitlines_with_ids

,FNODE,OBJECT_ID_x,geometry,TNODE
0,NO25019967,RL25020350,"LINESTRING (145305.631 166954.356, 145305.983 ...",NO250199672
1,NO250199672,RL25020350,"LINESTRING (145345.099 167120.247, 145350.687 ...",NO25020064
2,NO25020532,RL25020829,"LINESTRING (150482.760 174395.771, 150486.285 ...",NO250199673
3,NO250199673,RL25020829,"LINESTRING (152942.295 177058.223, 152961.803 ...",NO25020533


In [68]:
#splitlines_with_ids.to_file(r"data_transform\bxl_split_segments.shp")

### Get unique ids for the new split segments based on original string ID

In [69]:
def get_unique_ID(df, col):
    """ Get unique ID for each new split segment in a dataframe
    Assert that the number of unique IDs is equal to the number of split segments
     """
    # the new split lines need a new unique uniqueID value
    df['num_id'] = df.groupby(col).cumcount()+1
    df["new_string_id"] = df[col].astype(str) + '_' + df['num_id'].astype(str)

    #df = df.drop(columns=['num_id', col]).rename(columns={'new_string_id': col})
    #assert df[col].nunique() == len(df)

    return df

In [70]:
splitlines_OBJECTID = get_unique_ID(splitlines_with_ids, 'OBJECT_ID_x')

In [71]:
splitlines_merged = splitlines_OBJECTID.merge(bxl_water_df, left_on='OBJECT_ID_x', right_on='OBJECT_ID', how='left')
assert splitlines_merged.OBJECT_ID_x.all() == splitlines_merged.OBJECT_ID.all()

In [72]:
splitlines_merged.columns

Index(['FNODE_x', 'OBJECT_ID_x', 'geometry_x', 'TNODE_x', 'num_id',
       'new_string_id', 'OBJECTID', 'DFDD', 'RN_I_ID', 'REX', 'HYP', 'LOC',
       'FUN', 'NVS', 'LENGTH', 'TR', 'LONGPATH', 'CUM_LEN', 'PENTE', 'CGNELIN',
       'BEGLIFEVER', 'ENDLIFEVER', 'UPDAT_BY', 'UPDAT_WHEN', 'ERM_ID', 'MC',
       'MONOT_Z', 'LENGTH_GEO', 'INSPIRE_ID', 'thematicId', 'OBJECT_ID',
       'TNODE_y', 'STRAHLER', 'nameTxtInt', 'nameText', 'NEXTUPID',
       'NEXTDOWNID', 'FNODE_y', 'CatchID', 'PFAFSTETTE', 'geometry_y'],
      dtype='object')

In [73]:
splitlines_final = splitlines_merged.drop(['OBJECT_ID_x', 'OBJECT_ID', 'num_id', 'FNODE_y', 'TNODE_y', 'geometry_y'], axis=1)\
                                    .rename(columns={'new_string_id': 'OBJECT_ID', 'FNODE_x':'FNODE', 'TNODE_x':'TNODE', 'geometry_x': 'geometry'})
splitlines_final.head(2)

,FNODE,geometry,TNODE,OBJECT_ID,OBJECTID,DFDD,RN_I_ID,REX,HYP,LOC,...,LENGTH_GEO,INSPIRE_ID,thematicId,STRAHLER,nameTxtInt,nameText,NEXTUPID,NEXTDOWNID,CatchID,PFAFSTETTE
0,NO25019967,"LINESTRING (145305.631 166954.356, 145305.983 ...",NO250199672,RL25020350_1,20350,BH140,None,BE,1.0,44.0,...,2673.962568,EEA_EUHydro:RL25020350:2015-03-22,BEVL08_92,4.0,None,ZENNE I,RL25020250,RL25020355,25,None
1,NO250199672,"LINESTRING (145345.099 167120.247, 145350.687 ...",NO25020064,RL25020350_2,20350,BH140,None,BE,1.0,44.0,...,2673.962568,EEA_EUHydro:RL25020350:2015-03-22,BEVL08_92,4.0,None,ZENNE I,RL25020250,RL25020355,25,None


In [74]:
splitlines_final.columns

Index(['FNODE', 'geometry', 'TNODE', 'OBJECT_ID', 'OBJECTID', 'DFDD',
       'RN_I_ID', 'REX', 'HYP', 'LOC', 'FUN', 'NVS', 'LENGTH', 'TR',
       'LONGPATH', 'CUM_LEN', 'PENTE', 'CGNELIN', 'BEGLIFEVER', 'ENDLIFEVER',
       'UPDAT_BY', 'UPDAT_WHEN', 'ERM_ID', 'MC', 'MONOT_Z', 'LENGTH_GEO',
       'INSPIRE_ID', 'thematicId', 'STRAHLER', 'nameTxtInt', 'nameText',
       'NEXTUPID', 'NEXTDOWNID', 'CatchID', 'PFAFSTETTE'],
      dtype='object')

## Gather all linestrings into one dataset

In [75]:
def merge_segments_to_water(split_segments, split_segments_final, water_df, col):
    """This function merges segments to water polygons"""
    # drop the linestrings to be split and merge the df with split lines
    #water_final_str = water_final_str.astype({"VHAS": str}, errors='raise')
    
    split_segments = split_segments.astype({col: str}, errors='raise')
    split_segments_final = split_segments_final.astype({col: str}, errors='raise')
    water_df = water_df.astype({col: str}, errors='raise')

    assert split_segments_final[col].nunique() == split_segments_final['geometry'].nunique()
    assert water_df[col].nunique() == water_df['geometry'].nunique()

    print(len(split_segments_final))
    print(len(water_df))
    linestrings_to_drop = split_segments[col].to_list()
    print('linestrings_to_drop:', len(set(linestrings_to_drop)))

    print("linestrings to drop: ", len(linestrings_to_drop))
    water_df_trimmed = water_df[~water_df[col].isin(linestrings_to_drop)].reset_index(drop=True)
    water_df_drop = water_df[water_df[col].isin(linestrings_to_drop)]
    assert water_df_trimmed[col].nunique() == water_df_trimmed['geometry'].nunique()
    print("water df trimmed: ", len(water_df_trimmed))
    print("water df drop: ", len(water_df_drop))

    #merge the split lines with the original water lines
    #water_df_trimmed_merged = water_df_trimmed.merge(split_segments_line_ids, on=col, how='outer')
    merged_df = gpd.GeoDataFrame(pd.concat([split_segments_final, water_df_trimmed], ignore_index=True), geometry='geometry', crs=PROJ_CRS)
    print("merged df: ", len(merged_df))
    assert merged_df['geometry'].nunique() == merged_df[col].nunique()

    #assert merged_df['geometry'].nunique() == merged_df[col].nunique()
    return merged_df

In [76]:
segments_to_water = merge_segments_to_water(splitlines_df, splitlines_final, bxl_water_df, 'OBJECT_ID')

4
83
linestrings_to_drop: 2
linestrings to drop:  4
water df trimmed:  81
water df drop:  2
merged df:  85


In [77]:
segments_to_water['LENGTH_M'] = segments_to_water['geometry'].apply(lambda x: x.length)

In [78]:
segments_to_water

,FNODE,geometry,TNODE,OBJECT_ID,OBJECTID,DFDD,RN_I_ID,REX,HYP,LOC,...,INSPIRE_ID,thematicId,STRAHLER,nameTxtInt,nameText,NEXTUPID,NEXTDOWNID,CatchID,PFAFSTETTE,LENGTH_M
0,NO25019967,"LINESTRING (145305.631 166954.356, 145305.983 ...",NO250199672,RL25020350_1,20350,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25020350:2015-03-22,BEVL08_92,4.0,None,ZENNE I,RL25020250,RL25020355,25,None,170.520982
1,NO250199672,"LINESTRING (145345.099 167120.247, 145350.687 ...",NO25020064,RL25020350_2,20350,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25020350:2015-03-22,BEVL08_92,4.0,None,ZENNE I,RL25020250,RL25020355,25,None,607.114333
2,NO25020532,"LINESTRING (150482.760 174395.771, 150486.285 ...",NO250199673,RL25020829_1,20829,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25020829:2015-03-22,BEBR_SENNE_ZENNE,1.0,SENNE-ZENNE,ZENNE,None,RL25020839,25,None,3722.494935
3,NO250199673,"LINESTRING (152942.295 177058.223, 152961.803 ...",NO25020533,RL25020829_2,20829,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25020829:2015-03-22,BEBR_SENNE_ZENNE,1.0,SENNE-ZENNE,ZENNE,None,RL25020839,25,None,340.527147
4,NO25019914,"LINESTRING (149013.615 164578.480, 148988.678 ...",NO25019899,RL25020197,20197,BH140,None,BE,NaN,NaN,...,EEA_EUHydro:RL25020197:2020-03-30,None,1.0,None,None,None,RL25020250,25,None,2637.979056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NO25019899,"LINESTRING (146845.939 164060.517, 146844.029 ...",NO25019967,RL25020250,20250,BH140,None,BE,NaN,NaN,...,EEA_EUHydro:RL25020250:2020-03-30,None,2.0,None,ZENNE I,RL25020197,RL25020350,25,None,2126.278074
81,NO25025490,"LINESTRING (142626.592 166792.652, 142629.851 ...",NO25020016,RL25025986,25986,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25025986:2020-03-30,BEVL05_94,1.0,None,ZUUNBEEK,None,RL25020298,25,None,2454.485073
82,NO25020400,"LINESTRING (144164.378 173446.948, 144169.258 ...",NO25020401,RL25020700,20700,BH140,None,BE,NaN,NaN,...,EEA_EUHydro:RL25020700:2020-03-30,None,1.0,SENNE-ZENNE,ZENNE,None,RL25020701,25,None,6815.572412
83,NO25020325,"LINESTRING (154047.853 176496.857, 154045.599 ...",NO25020533,RL25020830,20830,BH140,None,BE,1.0,44.0,...,EEA_EUHydro:RL25020830:2015-03-22,BEVL11_91,3.0,None,WOLUWE,RL25020636,RL25020839,25,None,1380.416511


In [79]:
# segments_to_water.to_file(r"data_transform\bxl_water_PROCESSED.shp")

In [80]:
#split_segments_line_ids.to_file(r"C:\Workdir\Develop\TR_USECASE\data_transform\wal_split_segments.shp")